ERROR: Can't invoke 'meta.llama3-70b-instruct-v1:0'. Reason: An error occurred (InvalidSignatureException) when calling the InvokeModel operation: The request signature we calculated does not match the signature you provided. Check your AWS Secret Access Key and signing method. Consult the service documentation for details.


In [10]:
!pip install --upgrade boto3
!pip install python-dotenv

In [ ]:
import os
import json
import boto3
from botocore.exceptions import ClientError
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class BedrockClient:
    def __init__(self, region_name, model_id):
        self.client = boto3.client('bedrock-runtime', region_name=region_name)
        self.model_id = model_id
        self.dataset = self.load_dataset()[:5]  # Load only first 5 questions

    def load_dataset(self):
        with open('RepresentativeDataset.json', 'r') as f:
            return json.load(f)

    def create_prompt(self, question, options):
        prompt = f"Solve the following mathematical problem:\n\nQuestion: {question}\n"
        prompt += f"Options: {', '.join(options)}\n\n"
        prompt += "Provide your solution and clearly state your final answer at the end, prefixed with the option choice as an alphabet for eg ('a' if 1 ) as 'Final Answer: '."

        return f"""<begin_of_text>
<start_header_id>user<end_header_id>
{prompt}
<eot_id>
<start_header_id>assistant<end_header_id>
"""

    def invoke_model(self, question, options, max_tokens=1024, temperature=0.7):
        formatted_prompt = self.create_prompt(question, options)

        request_body = {
            "prompt": formatted_prompt,
            "max_gen_len": max_tokens,
            "temperature": temperature,
        }

        try:
            response = self.client.invoke_model(
                body=json.dumps(request_body),
                contentType='application/json',
                modelId=self.model_id
            )
            response_body = json.loads(response['body'].read().decode('utf-8'))
            return response_body.get('generation', '')
        except ClientError as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            return None

    def extract_model_answer(self, model_solution):
        match = re.search(r"Final Answer:\s*(\w+)", model_solution, re.IGNORECASE)
        if match:
            return match.group(1).lower()
        return None

    def process_questions(self):
        results = []
        correct_count = 0
        for problem in self.dataset:
            result = self.invoke_model(problem['question'], problem['options'])
            model_answer = self.extract_model_answer(result)
            is_correct = model_answer == problem['answer'].lower() if model_answer else False
            if is_correct:
                correct_count += 1

            results.append({
                'question': problem['question'],
                'model_solution': result,
                'model_answer': model_answer,
                'correct_solution': problem['solution'],
                'correct_answer': problem['answer'],
                'is_correct': is_correct
            })

        accuracy = correct_count / len(self.dataset) if self.dataset else 0
        return results, accuracy

def main():
    region_name = os.environ.get('AWS_REGION', 'us-east-1')
    model_id = os.environ.get('BEDROCK_MODEL_ID', 'meta.llama2-70b-chat-v1')

    client = BedrockClient(region_name, model_id)
    results, accuracy = client.process_questions()

    # Print results
    for i, result in enumerate(results, 1):
        print(f"\nQuestion {i}:")
        print(f"Q: {result['question']}")
        print(f"Model's solution:\n{result['model_solution']}")
        print(f"Model's extracted answer: {result['model_answer']}")
        print(f"Correct solution: {result['correct_solution']}")
        print(f"Correct answer: {result['correct_answer']}")
        print(f"Is correct: {result['is_correct']}")
        print("-" * 50)

    print(f"\nOverall Accuracy: {accuracy:.2%}")

if __name__ == "__main__":
    main()


Question 1:
Q: If \(\int\left(e^{2 x}+2 e^{x}-e^{-x}-1\right) e^{\left(e^{x}+e^{-x}\right)} d x=g(x) e^{\left(e^{x}+e^{-x}\right)}+c\), where \(c\) is a constant of integeration, then \(g(0)\) is equal to:
Model's solution:
Final Answer: a

The given problem is:

g(x) = e^(x + e^(-x)) + c

We need to find g(0).

g(0) = e^(0 + e^(-0)) + c
= e^(e^(-0)) + c
= e^(1) + c

So, g(0) is equal to e + c.

Now, the options are:

a) e
b) e^2
c) 1
d) 2

The correct answer is 'a' e.
Model's extracted answer: a
Correct solution: }
  \item (d) \(\int\left(e^{2 x}+2 e^{x}-e^{-x}-1\right) \cdot e^{\left(e^{x}+e^{-x}\right)} d x\)


\(I=\int\left(e^{2 x}+e^{x}-1\right) \cdot e^{\left(e^{x}+e^{-x}\right)} d x+\int\left(e^{x}-e^{-x}\right) e^{\left(e^{x}+e^{-x}\right)} d x\)

\(=\int e^{x}\left(e^{x}+1-e^{-x}\right) \cdot e^{\left(e^{x}+e^{-x}\right)} d x+e^{\left(e^{x}+e^{-x}\right)}\)

\(=\int\left(e^{x}-e^{-x}+1\right) e^{\left(e^{x}+e^{-x}+x\right)} d x+e^{\left(e^{x}+e^{-x}\right)}\)

Let \(e^{x}+e^{